# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902080


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/83 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/83 [00:05<03:30,  2.60s/it]

Rendering models:   6%|▌         | 5/83 [00:05<02:22,  1.83s/it]

Rendering models:   7%|▋         | 6/83 [00:06<02:12,  1.71s/it]

Rendering models:  10%|▉         | 8/83 [00:06<01:31,  1.22s/it]

...

Rendering models:  82%|████████▏ | 68/83 [00:09<00:00, 18.94it/s]

Rendering models:  86%|████████▌ | 71/83 [00:09<00:00, 20.01it/s]

Rendering models:  89%|████████▉ | 74/83 [00:09<00:00, 21.67it/s]

Rendering models:  96%|█████████▋| 80/83 [00:09<00:00, 25.67it/s]

equidad1                              0.173237
Linda_J._Berkel                       0.000296
not-logged-in-587a4e9ad373b8ff5983    0.001576
alochiatto                            0.000597
Thedunkmasta                          0.001857
Thedunkmasta                          0.001864
DrJackie                              0.025696
insert_name                           0.007809
not-logged-in-56b19a5e192cb0454662    0.001250
not-logged-in-ab1612bc18cd7aab85db    0.000200
msross                                0.000308
clizardo                              0.002827
mkirms                                0.000247
msross                                0.000545
not-logged-in-988154c28a4325297d39    0.003109
djcameron                             0.000607
mchaffin                              0.000299
atinkess                              0.002358
denzel.ayala                          0.001002
creinbol                              0.000635
cchen88                               0.048315
denzel.ayala 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())